In [3]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
import pandas as pd  
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import classification_report
from sklearn import metrics
import numpy as np
from sklearn.decomposition import PCA
from tensorflow.python.keras.models import Sequential
from keras import optimizers
from tensorflow.python.keras.layers import Dense, Activation
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle

Using TensorFlow backend.
C:\Users\Saloni\Anaconda3\envs\my_flask_env\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
data = pd.read_csv('maruti_data.csv')
#print(data)
x=data.iloc[212:2715,6:47]
y=data.iloc[212:2715,4]
print(x)
#print(y)
"""kpca = KernelPCA(kernel="rbf", gamma=10)
X_kpca = kpca.fit_transform(x)
#X_back = kpca.inverse_transform(X_kpca)
pca = PCA(n_components=36)"""
x_std = StandardScaler().fit_transform(x)
pca = PCA(n_components=36)
principalComponents = pca.fit_transform(x_std)
df = pd.DataFrame(data = principalComponents)
df.to_csv('pca.csv')
data = pd.read_csv('pca.csv')
#print(data)

         A/D line         ohlc  aroon up  aroon down         ATR       DMI+  \
212   3855691.322   544.100006        90          60   32.604990  23.384106   
213   3641551.029   564.600006        90          50   32.801065  21.584014   
214   3804072.420   557.899994        80          40   32.279560  22.103165   
215   3775698.830   566.537506        90          30   32.002451  24.340200   
216   4817659.139   571.237488        90          20   32.995134  22.083989   
217   4837345.430   579.449982        90          10   33.149055  20.411343   
218   5436782.947   582.125000        90           0   32.852694  21.081153   
219   5987620.716   590.187500        90           0   32.363216  19.871425   
220   6782626.208   576.375000        80           0   34.040846  17.542668   
221   6755954.655   589.862503        70           0   32.759359  16.926841   
222   6544708.314   577.924988        60          10   31.669405  16.258734   
223   6360857.868   562.875000        50           0

In [7]:
from pandas import Series
from statsmodels.tsa.stattools import adfuller
series = Series.from_csv('pca.csv')
X = series.values
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

C:\Users\Saloni\Anaconda3\envs\my_flask_env\lib\site-packages\pandas\core\series.py:3727: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


ADF Statistic: -8.190930
p-value: 0.000000
Critical Values:
	1%: -3.433
	5%: -2.863
	10%: -2.567


In [4]:
data = pd.read_csv('g_op11.csv')
df = pd.DataFrame(data)
headers = df.dtypes.index
print(headers)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '25', '26', '27',
       '28', '29', '30', '31', '32', '33', '34', '35', 'class'],
      dtype='object')


In [5]:
from sklearn.model_selection import train_test_split
X=data.iloc[:2501,0:33]
Y=data.iloc[:2501,33]
#print(X)
print(Y)

0        544.400024
1        557.049988
2        561.400024
3        565.150024
4        585.849976
5        579.349976
6        587.500000
7        595.750000
8        589.950012
9        589.650024
10       573.900024
11       559.599976
12       539.750000
13       519.400024
14       515.200012
15       537.650024
16       520.549988
17       544.450012
18       568.299988
19       567.000000
20       583.750000
21       602.000000
22       577.049988
23       577.700012
24       595.200012
25       608.400024
26       632.299988
27       613.849976
28       628.799988
29       622.099976
           ...     
2469    7336.250000
2470    7357.950195
2471    7208.399902
2472    7068.850098
2473    7045.399902
2474    7040.600098
2475    6513.399902
2476    6511.049805
2477    6526.950195
2478    6548.250000
2479    6641.149902
2480    6962.299805
2481    6988.200195
2482    7093.600098
2483    7183.399902
2484    7302.549805
2485    7134.250000
2486    7177.899902
2487    7162.500000


In [6]:
ranks = {}
# Create our function which stores the feature rankings to the ranks dictionary
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    print(dict(zip(names, ranks)))
    return dict(zip(names, ranks))
lr = LinearRegression()
lr.fit(X,Y)
#stop the search when only the last feature is left
rfe = RFE(lr, n_features_to_select=1)
rfe.fit(X,Y)
ranks["RFE"] = ranking(list(map(float, rfe.ranking_)), headers, order=-1) 
print ("Features sorted by their rank:")
print (sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), headers)))

{'0': 0.91, '1': 0.47, '2': 0.03, '3': 0.31, '4': 0.56, '5': 0.0, '6': 0.16, '7': 0.06, '8': 0.25, '10': 0.34, '11': 0.66, '12': 0.12, '13': 0.62, '14': 0.19, '15': 0.72, '16': 0.69, '17': 0.53, '18': 0.5, '19': 0.41, '20': 0.09, '21': 0.28, '22': 0.59, '25': 0.38, '26': 0.44, '27': 0.81, '28': 0.75, '29': 0.88, '30': 0.78, '31': 0.84, '32': 0.22, '33': 0.94, '34': 1.0, '35': 0.97}
Features sorted by their rank:
[(1, '34'), (2, '35'), (3, '33'), (4, '0'), (5, '29'), (6, '31'), (7, '27'), (8, '30'), (9, '28'), (10, '15'), (11, '16'), (12, '11'), (13, '13'), (14, '22'), (15, '4'), (16, '17'), (17, '18'), (18, '1'), (19, '26'), (20, '19'), (21, '25'), (22, '10'), (23, '3'), (24, '21'), (25, '8'), (26, '32'), (27, '14'), (28, '6'), (29, '12'), (30, '20'), (31, '7'), (32, '2'), (33, '5')]


In [19]:
from sklearn.model_selection import train_test_split
data = pd.read_csv('g_op11.csv')
train_x=data.iloc[:2488,0:33]
train_y=data.iloc[:2488,33]
test_x=data.iloc[2488:2501,0:33]
test_y=data.iloc[2488:2501,33]
print(test_y)
#train_x, test_x, train_y, test_y = train_test_split(data.iloc[:,1:11], data.iloc[:,12], test_size=0.2, random_state=None, shuffle=False)

2488    7017.500000
2489    6930.049805
2490    6842.700195
2491    6804.549805
2492    6849.899902
2493    6801.549805
2494    6912.399902
2495    6924.899902
2496    6904.299805
2497    6968.500000
2498    6829.700195
Name: class, dtype: float64


In [20]:
def svr_model(train_x, train_y, test_x):
    from sklearn.svm import SVR
    import numpy as np

    clf = SVR(C=1.0, epsilon=2.0, gamma=5e-8)
    clf.fit(train_x, train_y)
    predicted_y = clf.predict(test_x)
    print(predicted_y)
    return predicted_y

In [21]:
predict_svr=svr_model(train_x, train_y, test_x)

[1730.0812001  1730.08083621 1730.08069739 1730.08060774 1730.08058603
 1730.08046916 1730.08047441 1730.08050773 1730.0803986  1730.0804605
 1730.08022658]


In [22]:
def random_forest_reg(train_x, train_y, test_x):
    from sklearn.ensemble import RandomForestRegressor
    import numpy as np
    rfr = RandomForestRegressor(max_depth=5, random_state=0)
    rfr.fit(train_x, train_y)
    rfr_predicted_y = rfr.predict(test_x)
    return rfr_predicted_y


In [23]:
predict_rf=random_forest_reg(train_x, train_y, test_x)


In [24]:
from sklearn.metrics import mean_squared_error
from math import sqrt


def error_function(test_y, predicted_y):
    import numpy as np
    from math import sqrt
    test = list(test_y)
    MAPE=[(test[i]-predicted_y[i])/test[i] for i in range(len(test))]
    MAPE1=abs((sum(MAPE)/len(test)))*100
    MAE=abs((sum(MAPE)/len(test)))
    rRMSE=[((test[i]-predicted_y[i])/test[i])**2 for i in range(len(test))]
    rRMSE=sqrt(sum(rRMSE)/len(test))
    return MAPE1,rRMSE,MAE

In [25]:
def gradient_boost(train_x, train_y, test_x):
    clf = GradientBoostingRegressor(loss='quantile',learning_rate=0.001, n_estimators=20, max_features='log2', min_samples_split=2, max_depth=1)
    clf.fit(train_x, train_y)
    predicted_y = clf.predict(test_x)
    return predicted_y

In [26]:
from sklearn.ensemble import GradientBoostingRegressor 
predict_gr=gradient_boost(train_x, train_y, test_x)

In [27]:
print("\t MAPE\t\tRMSE\t\tMAE")
print("SVR:",error_function(test_y, predict_svr))
print("Random_forest:",error_function(test_y, predict_rf))
print("gradient_boost:",error_function(test_y,predict_gr))

	 MAPE		RMSE		MAE
SVR: (74.8863396926932, 0.7488672897110313, 0.7488633969269319)
Random_forest: (7.479417512496202, 0.07848501828144756, 0.07479417512496202)
gradient_boost: (14.473233576319103, 0.14513146692355347, 0.14473233576319103)
